## 스레드 시작과 정지

In [1]:
from threading import Thread
import time

class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print("T-minus", n)
            n -= 1
            time.sleep(5)

c = CountdownTask()
t = Thread(target=c.run, args=(10,))
t.start()

time.sleep(20)
print('About to terminate')
c.terminate()
t.join()
print('Terminated')

T-minus 10
T-minus 9
T-minus 8
T-minus 7
About to terminate
Terminated


## 스레드가 시작했는지 판단하기

In [2]:
from threading import Thread, Event
import time

def countdown(n, started_evt):
    print("countdown starting")
    started_evt.set() 
    while n > 0:
        print("T-minus", n)
        n -= 1
        time.sleep(5)

started_evt = Event()

print("Launching countdown")
t = Thread(target=countdown, args=(10,started_evt))
t.start()

started_evt.wait()
print("countdown is running")

Launching countdown
countdown starting
T-minuscountdown is running 10

T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


In [1]:
import threading
import time

class PeriodicTimer:
    def __init__(self, interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()

    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True
        t.start()

    def run(self):
        while True:
            time.sleep(self._interval)
            with self._cv:
                 self._flag ^= 1
                 self._cv.notify_all()

    def wait_for_tick(self):
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()

ptimer = PeriodicTimer(5)
ptimer.start()

def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print("T-minus", nticks)
        nticks -= 1

def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print("Counting", n)
        n += 1

threading.Thread(target=countdown, args=(10,)).start()
threading.Thread(target=countup, args=(5,)).start()

T-minus 10
Counting 0
T-minusCounting 9
 1
T-minusCounting  8
2
CountingT-minus 3
 7
T-minusCounting 6
 4
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


In [2]:
import threading
import time

def worker(n, sema):
    sema.acquire()
    print("Working", n)

sema = threading.Semaphore(0)
nworkers = 10
for n in range(nworkers):
    t = threading.Thread(target=worker, args=(n, sema,))
    t.daemon=True
    t.start()

print('About to release first worker')
time.sleep(5)
sema.release()
time.sleep(1)
print('About to release second worker')
time.sleep(5)
sema.release()
time.sleep(1)
print('Goodbye')

About to release first worker
Working 0
About to release second worker
Working 1
Goodbye


## 스레드간 통신

In [5]:
from queue import Queue
from threading import Thread
import time

_sentinel = object()

def producer(out_q):
    n = 10
    while n > 0:
        out_q.put(n)
        time.sleep(2)
        n -= 1
    out_q.put(_sentinel)

def consumer(in_q):
    while True:
        data = in_q.get()

        if data is _sentinel:
            in_q.put(_sentinel)
            break

        print('Got:', data)
    print('Consumer shutting down')

if __name__ == '__main__':
    q = Queue()
    t1 = Thread(target=consumer, args=(q,))
    t2 = Thread(target=producer, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

Got: 10
Got: 9
Got: 8
Got: 7
Got: 6
Got: 5
Got: 4
Got: 3
Got: 2
Got: 1
Consumer shutting down


In [6]:
import heapq
import threading
import time

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._count = 0
        self._cv = threading.Condition()
    def put(self, item, priority):
        with self._cv:
            heapq.heappush(self._queue, (-priority, self._count, item))
            self._count += 1
            self._cv.notify()

    def get(self):
        with self._cv:
            while len(self._queue) == 0:
                self._cv.wait()
            return heapq.heappop(self._queue)[-1]

def producer(q):
    print('Producing items')
    q.put('C', 5)
    q.put('A', 15)
    q.put('B', 10)
    q.put('D', 0)
    q.put(None, -100)

def consumer(q):
    time.sleep(5)
    print('Getting items')
    while True:
        item = q.get()
        if item is None:
            break
        print('Got:', item)
    print('Consumer done')

if __name__ == '__main__':
    q = PriorityQueue()
    t1 = threading.Thread(target=producer, args=(q,))
    t2 = threading.Thread(target=consumer, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

Producing items
Getting items
Got: A
Got: B
Got: C
Got: D
Consumer done
